## 對文章標題做encoding，把每篇文章弄成同維度，可以做CNN的前處理

## 後面model是做conv1d+lstm

In [44]:
import jieba
import jieba.analyse
import csv
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import re
from tqdm import tqdm

In [ ]:
company = '台積電'

## 對股票價格處理

In [ ]:
stock2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv", encoding="utf-8")
stock2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv", encoding="utf-8")
stock2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv", encoding="utf-8")
df_stock = pd.concat([stock2018,stock2017,stock2016],axis=0, ignore_index=True)
stock_tgg = df_stock[df_stock['證券代碼'].str.contains(company)]
stock_tgg = stock_tgg.reset_index(drop=True)
stock_tgg['年月日']=pd.to_datetime(stock_tgg['年月日'])
stock_tgg['收盤價(元)']=stock_tgg['收盤價(元)'].astype(str).str.replace(',','').astype('float64')

In [ ]:
stock_tgg['highpriceshift'] = stock_tgg['收盤價(元)'].shift(periods=1)
stock_tgg['div']=stock_tgg['highpriceshift']-stock_tgg['收盤價(元)']
stock_tgg = stock_tgg[1:]
stock_tgg = stock_tgg.reset_index(drop=True)
stock_tgg['label'] = '平'
stock_tgg.loc[stock_tgg['div']>0.00, 'label'] = '漲'
stock_tgg.loc[stock_tgg['div']<0.00, 'label'] = '跌'
stock_tgg.head()

In [ ]:
tgg_high = stock_tgg[stock_tgg['label'].str.contains('漲')]
tgg_low = stock_tgg[stock_tgg['label'].str.contains('跌')]
tgg_even = stock_tgg[stock_tgg['label'].str.contains('平')]
print("漲的天數: %d" % len(tgg_high))
print("跌的天數: %d" % len(tgg_low))
print("平的天數: %d" % len(tgg_even))
print("全部的天數: %d" % len(stock_tgg))

## 對文章處理

In [45]:
news=pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
news['title'].fillna(value='N/A', inplace=True)
news['content'].fillna(value='N/A', inplace=True)
# tnews=news[news['title'].str.contains(company)]
tnews=news[news['content'].str.contains('台積電|台積|張忠謀')]
tnews=tnews.reset_index(drop=True)
tnews['post_time']=pd.to_datetime(tnews['post_time'])
print("台積電文章: %d" % len(tnews))

台積電文章: 18423


In [ ]:
for day in tqdm(range(len(tnews))):
    targetdate=tnews.loc[day,'post_time'].date()
    for day1 in range(len(stock_tgg)):
        try:
            if targetdate==stock_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=stock_tgg.loc[day1,'label']
            elif (targetdate+timedelta(1))==stock_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=stock_tgg.loc[day1+1,'label']
            elif (targetdate+timedelta(2))==stock_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=stock_tgg.loc[day1+1,'label']
            elif (targetdate+timedelta(3))==stock_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=stock_tgg.loc[day1+1,'label']
            elif (targetdate+timedelta(4))==stock_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=stock_tgg.loc[day1+1,'label']
        except:
            continue

In [ ]:
tnews['label'].fillna(value='N/A', inplace=True)
tnews1=tnews[tnews['label'].str.contains('漲')]
tnews2=tnews[tnews['label'].str.contains('跌')]
tnews3=tnews[tnews['label'].str.contains('平')]
tnews4=tnews[tnews['label'].str.contains('N/A')]
print("漲的文章:", len(tnews1))
print("跌的文章:", len(tnews2))
print("平的文章:", len(tnews3))
print("N/A的文章:", len(tnews4))
print("全部的文章:", len(tnews))

## 讀取整理好的資料

In [46]:
tnews = pd.read_csv("bda2019_dataset/stock_content_tgg.csv",encoding='utf-8')
tnews['post_time']=pd.to_datetime(tnews['post_time'])
tnews = tnews.drop(columns=['Unnamed: 0','id','p_type','s_name','s_area_name','author','page_url'])
tnews.head()

,post_time,title,content,label
0,2016-01-01 09:05:00,工商頭條~4大基金封關加碼30億 2016年台股漲像NIKE,2015年台股昨（31）日封關，四大基金在尾盤大舉加碼買進台積電、國泰金等權值股，總計投入約...,跌
1,2016-01-01 09:27:00,《各報要聞》2016年台股漲像NIKE,2015年台股昨（31）日封關，四大基金在尾盤大舉加碼買進台積電、國泰金等權值股，總計投入約...,跌
2,2016-01-01 11:46:00,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,跌
3,2016-01-02 11:12:00,iPhone新4吋 傳7大特色先睹為快,（中央社記者鍾榮峰台北2016年1月2日電）蘋果(Apple)新4吋iPhone萬眾矚目，各...,跌
4,2016-01-03 08:59:00,《各報要聞》外資歸隊，台股帶量拚季線,台股上周封關日集中及櫃買兩市場同步收紅，本周為105年元月首周，市場聚焦新掛牌股、外資歸隊、...,跌


## 斷詞前處理

In [47]:
stopwords = list()
with open('bda2019_dataset/stopword.txt', 'r', encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [48]:
tnews['label'].fillna(value='N/A', inplace=True)
tnews=tnews[tnews['label'].str.contains('漲|跌')]
tnews=tnews.reset_index(drop=True)
len(tnews)

16724

In [49]:
tnews.loc[tnews['label']=='漲', 'num_label'] = 1
tnews.loc[tnews['label']=='跌', 'num_label'] = 0

In [50]:
jieba.set_dictionary('bda2019_dataset/dict.idkrsi.txt')
corpus = list()
for num in range(len(tnews)):
    content=tnews.loc[num,'title']
    content = re.sub('[a-zA-Z0-9\W]', '', content)
    seglist=jieba.cut(content.replace('\n','').replace('，',' '),cut_all=False)
    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
    cutcontent=' '.join(seglist)
    corpus.append(cutcontent)
tnews["cutcontent"]=corpus
tnews.head()

Building prefix dict from /home/daniel/code/finance/bda2019_dataset/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.u2e58f25a25b46fee917713c08b4e0329.cache
Loading model cost 0.355 seconds.
Prefix dict has been built succesfully.


,post_time,title,content,label,num_label,cutcontent
0,2016-01-01 09:05:00,工商頭條~4大基金封關加碼30億 2016年台股漲像NIKE,2015年台股昨（31）日封關，四大基金在尾盤大舉加碼買進台積電、國泰金等權值股，總計投入約...,跌,0.0,工商 頭條 基金 封關 加碼 億年 台股 漲像
1,2016-01-01 09:27:00,《各報要聞》2016年台股漲像NIKE,2015年台股昨（31）日封關，四大基金在尾盤大舉加碼買進台積電、國泰金等權值股，總計投入約...,跌,0.0,各報 要聞 台股 漲像
2,2016-01-01 11:46:00,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,跌,0.0,台積電 下跌 美元 跌幅 折 台股 元
3,2016-01-02 11:12:00,iPhone新4吋 傳7大特色先睹為快,（中央社記者鍾榮峰台北2016年1月2日電）蘋果(Apple)新4吋iPhone萬眾矚目，各...,跌,0.0,新 吋 傳大 特色 先睹為快
4,2016-01-03 08:59:00,《各報要聞》外資歸隊，台股帶量拚季線,台股上周封關日集中及櫃買兩市場同步收紅，本周為105年元月首周，市場聚焦新掛牌股、外資歸隊、...,跌,0.0,各報 要聞 外資 歸隊 台股 帶量 拚 季線


## 對文章做day instacne (同一天的文章同一個label)(一天超過7篇文章會換下一個label)

In [51]:
tnews.loc[0,'day_instance'] = 0
day_instance=0
count=1
article_count = list()  # day_instance數
for day in range(1,len(tnews)):
    if tnews.loc[day,'post_time'].date() == tnews.loc[day-1,'post_time'].date():  # 判斷是不是同一天
        if count<7:                                          # 如果一天文章超過7篇，每7篇一個day_instance
            tnews.loc[day,'day_instance'] = day_instance
            count+=1
        else:
            article_count.append(count)                      # 超過7篇重新設一個day_instance
            count=1
            day_instance+=1
            tnews.loc[day,'day_instance'] = day_instance
    else:
        article_count.append(count)                          # 日期不一樣就重設一個day_instance
        count=1
        day_instance+=1
        tnews.loc[day,'day_instance'] = day_instance
article_count.append(count)                                  # 把最後一天的day_instance加進來
tnews.head()

,post_time,title,content,label,num_label,cutcontent,day_instance
0,2016-01-01 09:05:00,工商頭條~4大基金封關加碼30億 2016年台股漲像NIKE,2015年台股昨（31）日封關，四大基金在尾盤大舉加碼買進台積電、國泰金等權值股，總計投入約...,跌,0.0,工商 頭條 基金 封關 加碼 億年 台股 漲像,0.0
1,2016-01-01 09:27:00,《各報要聞》2016年台股漲像NIKE,2015年台股昨（31）日封關，四大基金在尾盤大舉加碼買進台積電、國泰金等權值股，總計投入約...,跌,0.0,各報 要聞 台股 漲像,0.0
2,2016-01-01 11:46:00,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,跌,0.0,台積電 下跌 美元 跌幅 折 台股 元,0.0
3,2016-01-02 11:12:00,iPhone新4吋 傳7大特色先睹為快,（中央社記者鍾榮峰台北2016年1月2日電）蘋果(Apple)新4吋iPhone萬眾矚目，各...,跌,0.0,新 吋 傳大 特色 先睹為快,1.0
4,2016-01-03 08:59:00,《各報要聞》外資歸隊，台股帶量拚季線,台股上周封關日集中及櫃買兩市場同步收紅，本周為105年元月首周，市場聚焦新掛牌股、外資歸隊、...,跌,0.0,各報 要聞 外資 歸隊 台股 帶量 拚 季線,2.0


In [52]:
# 算一天大於4篇文章的天數
article_over = 0
delete_day = list()   #小於預定天數的day_instance 的 index
for i in range(len(article_count)):
    if article_count[i]>=4:
        article_over+=1
    else:
        delete_day.append(i)
print('總共文章天數:%d' % len(article_count))
print('總共大於4篇文章day_instance:%d' % article_over)

總共文章天數:2840
總共大於4篇文章day_instance:2352


In [53]:
# 把一天文章小於4篇的刪掉
tnews['day_instance'] = tnews['day_instance'].astype(int)
for i in range(len(tnews)):
    if tnews.loc[i,'day_instance'] in delete_day:
        tnews = tnews.drop([i])
tnews = tnews.reset_index(drop=True)

In [54]:
tnews.head(20)

,post_time,title,content,label,num_label,cutcontent,day_instance
0,2016-01-04 07:24:00,經濟頭條~台積16奈米 完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0.0,經濟 頭條 台積 奈米 完勝 三星,3
1,2016-01-04 07:24:00,工商頭條~千億科技預算 張善政喊加碼,行政院周五（8日）將召開毛內閣任內最後一次科技會報，會中將向張忠謀等外部委員報告106年度科...,跌,0.0,工商 頭條 千億 科技 預算 張善政 喊 加碼,3
2,2016-01-04 07:34:00,《半導體》i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在今年下半年登場，根據外電消息，蘋果自行設計的A10應...,跌,0.0,半導體 捎 單 有影 台積電 營運 逐季旺,3
3,2016-01-04 07:37:00,《各報要聞》千億科技預算，張善政喊加碼,行政院周五（8日）將召開毛內閣任內最後一次科技會報，會中將向張忠謀等外部委員報告106年度科...,跌,0.0,各報 要聞 千億 科技 預算 張善政 喊 加碼,3
4,2016-01-04 07:56:00,工商時報晨訊,"◆千億科技預算，張善政喊加碼，主計總處初步匡列1,061億、年增3%，可能再調高。<BR> ...",跌,0.0,工商時報 晨訊,3
5,2016-01-04 07:56:00,經濟日報晨訊,◆台積電16奈米，完勝三星，製程強壓對手的14奈米，蘋果A10代工大單可望入袋，三星暫緩擴廠...,跌,0.0,經濟日報 晨訊,3
6,2016-01-04 08:00:00,經濟日報頭條：台積16奈米，完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0.0,經濟日報 頭條 台積 奈米 完勝 三星,3
7,2016-01-04 08:01:00,工商時報頭條：千億科技預算，張善政喊加碼,行政院週五(8日)將召開毛內閣任內最後一次科技會報，會中將向張忠謀等外部委員報告106年度科...,跌,0.0,工商時報 頭條 千億 科技 預算 張善政 喊 加碼,4
8,2016-01-04 08:01:00,工商證券頭條：i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在2016年下半年登場，根據外電消息，蘋果自行設計的A...,跌,0.0,工商 證券 頭條 捎 單 有影 台積電 營運 逐季旺,4
9,2016-01-04 08:24:00,【Y早報】台積16奈米完勝三星,開盤日9:00出刊<BR>美股連2跌，道瓊合計下跌295點<BR>台股面臨補跌，國泰金重登金...,跌,0.0,早報 台積 奈米 完勝 三星,4


In [55]:
print('篩選過後的文章 %d' % len(tnews))

篩選過後的文章 15796


## 對文字encoding

In [56]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.preprocessing.text import Tokenizer

In [57]:
X = tnews['cutcontent'].to_numpy()
# Y = tnews['num_label'].to_numpy()

# le = LabelEncoder()
# Y = le.fit_transform(Y).reshape(-1,1)

In [58]:
# 對 Y 做lable，[1,0]是跌，[0,1]是漲
# ohe = OneHotEncoder()
# Y_label = ohe.fit_transform(Y).toarray()
# Y_label

In [59]:
max_words = 50000
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(tnews['cutcontent'])

for i, iterm in enumerate(tok.word_index.items()):
    if i < 10:
        print(iterm)
    else:
        break
print("=============================")
for i, iterm in enumerate(tok.word_counts.items()):
    if i < 10:
        print(iterm)
    else:
        break
word_index = tok.word_index
print('Found %s unique tokens.' % len(word_index))
# word_index就是根據cutcontent內容彙整出來的切字跟代表那個字的token number (每個字的dict)

('台股', 1)
('台股盤', 2)
('台積電', 3)
('外資', 4)
('盤', 5)
('解析', 6)
('市場', 7)
('中', 8)
('期貨', 9)
('晨訊', 10)
('經濟', 125)
('頭條', 303)
('台積', 451)
('奈米', 245)
('完勝', 9)
('三星', 100)
('工商', 53)
('千億', 30)
('科技', 136)
('預算', 16)
Found 10270 unique tokens.


In [60]:
# # 每篇文章title斷詞變成編碼的樣子
X_seq = tok.texts_to_sequences(X)
X_seq[0]

[171, 75, 44, 105, 1396, 208]

## Embedding layer設置

In [61]:
import os
import sys

embeddings_index = {}
f = open('bda2019_dataset/cc.zh.300.vec', 'r', encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2000001it [01:49, 18346.84it/s]

Found 2000000 word vectors.


## 此時，我們可以根據得到的字典生成上文所定義的詞向量矩陣

In [62]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM))       #預設一個全部都是0的matrix
for word, i in word_index.items():                                    #dict的index從1開始
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector         #將找到的embedding vector丟到他位置的matrix, 如果找不到一樣維持0
print(embedding_matrix.shape)
print(embedding_matrix)
# embedding_matrix就是把word_index裡面的每個字所代表word embedding對應變成一個matrix (每個字的word embedding)

(10271, 300)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.0152      0.15899999  0.2421     ...  0.16689999  0.38710001
  -0.20720001]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.0209     -0.0256      0.38909999 ... -0.0069     -0.0683
   0.13339999]
 [ 0.0429      0.072       0.40540001 ... -0.1552     -0.0955
  -0.0058    ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [63]:
# embedding_matrix[322]          #隨機抽查一個看看有沒有存進去

## 將每一篇文章每個字embedding加起來，並將每篇文章串起來

In [64]:
tnews['title_embedding'] = 'NA'      #先設一個空值的column
tnews.head()

,post_time,title,content,label,num_label,cutcontent,day_instance,title_embedding
0,2016-01-04 07:24:00,經濟頭條~台積16奈米 完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0.0,經濟 頭條 台積 奈米 完勝 三星,3,NA
1,2016-01-04 07:24:00,工商頭條~千億科技預算 張善政喊加碼,行政院周五（8日）將召開毛內閣任內最後一次科技會報，會中將向張忠謀等外部委員報告106年度科...,跌,0.0,工商 頭條 千億 科技 預算 張善政 喊 加碼,3,NA
2,2016-01-04 07:34:00,《半導體》i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在今年下半年登場，根據外電消息，蘋果自行設計的A10應...,跌,0.0,半導體 捎 單 有影 台積電 營運 逐季旺,3,NA
3,2016-01-04 07:37:00,《各報要聞》千億科技預算，張善政喊加碼,行政院周五（8日）將召開毛內閣任內最後一次科技會報，會中將向張忠謀等外部委員報告106年度科...,跌,0.0,各報 要聞 千億 科技 預算 張善政 喊 加碼,3,NA
4,2016-01-04 07:56:00,工商時報晨訊,"◆千億科技預算，張善政喊加碼，主計總處初步匡列1,061億、年增3%，可能再調高。<BR> ...",跌,0.0,工商時報 晨訊,3,NA


In [ ]:
title_numpy = np.zeros((300))
# all_title_numpy = np.zeros((1,300)) #先創造一個沒屁用的numpy array，方便串
for i in range(len(X_seq)):
    for j in range(len(X_seq[i])):
        target_token = X_seq[i][j]
        target_embedding = embedding_matrix[target_token]                    # 找到目標字的word embedding
        title_numpy = title_numpy + np.array(target_embedding)               # title_numpy = np.add(title_numpy, target_embedding)
    tnews['title_embedding'][i]= title_numpy

In [ ]:
tnews.head()

In [ ]:
# tnews.loc[300,'title_embedding']       # 抽查看有沒有放進dataframe

In [ ]:
# 把同一天instacne變成一個二維的矩陣
title_set = list()
y_set = list()
title_2d = np.zeros((7,300))
dayinst_count=0                                                          # 用來判斷要放在(7,300)的哪一個位置的count
for i in range(len(tnews)):
    if i ==0:                                                            # 先判斷是不是第一筆資料 
        title_2d[0] = tnews.loc[i,'title_embedding']
        dayinst_count+=1
    else:                                                                # 如果這筆資料跟前一筆的day_instance一樣
        if tnews.loc[i,'day_instance'] == tnews.loc[i-1,'day_instance']: # 那就把它存到同一個二維矩陣
            title_2d[dayinst_count] = tnews.loc[i,'title_embedding']
            dayinst_count+=1
        else:                                                            # 如果這筆資料跟前一筆的day_instance不一樣
            title_set.append(title_2d)                                   # 先封存前一筆day_instance建立好的矩陣
            y_set.append(tnews.loc[i-1,'num_label'])                     # 封存 Y label
            title_2d = np.zeros((7,300))                                 # 重創一個(7,300)的全0矩陣
            dayinst_count = 0                                            # 重設dayinst count
            title_2d[dayinst_count] = tnews.loc[i,'title_embedding']     # 把它存到同一個二維矩陣
            dayinst_count+=1
title_set.append(title_2d)                                               # 把最後一筆二維矩陣封存
y_set.append(tnews.loc[len(tnews)-1,'num_label'])                        # 把最後一筆 Y label封存
y_set = np.array(y_set)
title_set = np.array(title_set)                                          # list轉numpy
print('X筆數',len(title_set))
print('Y筆數',len(y_set))
print(title_set.shape)

In [ ]:
# 把tilte_set reshape
title_set = title_set.reshape((2352, 7, 300, 1))
title_set.shape

In [ ]:
up = 0
down = 0
for i in y_set:
    if i == 0.0:
        down+=1
    else:
        up+=1
print('漲的訓練資料筆數: %d' % up)
print('跌的訓練資料筆數: %d' % down)

In [ ]:
print(type(title_set),type(y_set))

In [ ]:
# [1,0]是跌，[0,1]是漲
le = LabelEncoder()
y_set = le.fit_transform(y_set).reshape(-1,1)
ohe = OneHotEncoder()
Y_label = ohe.fit_transform(y_set).toarray()
Y_label

In [ ]:
Y_label.shape

In [ ]:
# 切資料
from sklearn import model_selection
validation_size = 0.2
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(
    title_set, Y_label, test_size=validation_size, random_state=seed)

print(X_train.shape)
print(Y_train.shape)
print(X_validation.shape)
print(Y_validation.shape)

## Model (兩個Model都可以，第一個model優點是可以把維度拉出再做處理)

In [ ]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D, Flatten 
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, TimeDistributed, InputLayer
from keras.optimizers import RMSprop, adam
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [ ]:
visible = Input(shape=(7,300,1))
conv1 = TimeDistributed(Conv1D(5, kernel_size=150, padding='same', activation='relu'))(visible)
pool1 = TimeDistributed(MaxPooling1D(pool_size=2))(conv1)
flat = TimeDistributed(Flatten())(pool1)
lstm1 = LSTM(256, dropout=0.2, return_sequences=True)(flat)
lstm2 = LSTM(128, dropout=0.2)(lstm1)
hidden1 = Dense(64, activation='relu')(lstm2)
hidden2 = Dropout(0.2)
output = Dense(2, activation='softmax')(hidden1)
model1 = Model(inputs=visible, outputs=output)
model1.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
print(model1.summary())

In [ ]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=5,
                                 kernel_size=3,
                                 padding='same',
                                 activation='relu'), input_shape=(7,300,1)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(64, activation='relu', name='dense1'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax', name='dense2'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
model_fit = model1.fit(X_train,Y_train,batch_size=32,epochs=20,
                      validation_data=(X_validation, Y_validation))




# train_history=model.fit(x=title_set, 
#                         y=Y_label,validation_split=0.2, 
#                         epochs=10, batch_size=32,verbose=1)